In [1]:
import investpy as ip
import yahooquery as yq
from pandas_datareader import data as pdr
import pandas as pd
from tqdm import trange
import yfinance as yf


In [2]:
df = pd.read_excel('./datas/MSCIeurope.xlsx')
df.head()

,Unnamed: 0,Ticker,Ticker Yahoo,Nom,Secteur,Equity class,Market Share,Weights,Valeur notionnelle,Actions,ISIN,Currency,Environment Score,Social Score,Governance Score,Total Score
0,0,NESN,NSRGY,NESTLE SA,Biens de consommation de base,Actions,"EUR 48 274 991,14",4.67,48274991.14,420838,CH0038863350,CHF,NaN,NaN,NaN,NaN
1,1,ASML,ASML,ASML HOLDING NV,Technologie de l'informatioon,Actions,"EUR 34 906 428,20",3.38,34906428.20,60802,NL0010273215,EUR,NaN,NaN,NaN,NaN
2,2,ROG,ROG.SW,ROCHE HOLDING PAR AG,Santé,Actions,"EUR 33 188 774,68",3.21,33188774.68,104989,CH0012032048,CHF,2.48,11.54,9.52,23.53
3,3,MC,MC.PA,LVMH,Biens de consommation cycliques,Actions,"EUR 29 028 191,40",2.81,29028191.40,41457,FR0000121014,EUR,0.76,5.93,5.67,12.37
4,4,NOVOB,NVO,NOVO NORDISK CLASS B,Santé,Actions,"EUR 28 242 417,87",2.73,28242417.87,247375,DK0060534915,DKK,NaN,NaN,NaN,NaN


### get yahoo ticker

In [2]:
def find_ticker(isin):
    try:
        company_name = ip.stocks.search_stocks(by='isin', value=str(isin))
        company_name = company_name["name"][0].split(' ')[0]
        symbol = yq.search(company_name)["quotes"][0]["symbol"]
    except:
        symbol = None
    return symbol

In [ ]:
for i in range(len(df)):
    df.loc[i, "Ticker Yahoo"] = find_ticker(df.loc[i, "ISIN"])
df = df[['Ticker', 'Ticker Yahoo', 'Nom', 'Secteur', 'Equity class', 'Market Share', 'Weights',
       'Valeur notionnelle', 'Actions', 'ISIN', 'Currency', ]] 

In [10]:
# total weight
print(df["Weights"].sum())
# lost weights
print(df.loc[(df['Ticker Yahoo'].isna()), 'Weights'].sum())

99.98999999999998
6.82


### get ESG scores

In [11]:
esg_scores = pd.DataFrame(columns = ['Ticker Yahoo', 'Environment Score', 'Social Score', 'Governance Score', 'Total Score'], index = range(len(df)))
# trange = tqdm(range(len(df)))

for i in trange(len(df)):
    ticker = df.loc[i, 'Ticker Yahoo']
    try:
        sus = yf.Ticker(ticker).sustainability
        scores = sus.loc[['environmentScore','socialScore','governanceScore','totalEsg'],'Value']
        esg_scores.loc[i] = [ticker, scores[0], scores[1], scores[2], scores[3]]
    except:
        pass
esg_scores


100%|██████████| 357/357 [09:28<00:00,  1.59s/it]


,Ticker Yahoo,Environment Score,Social Score,Governance Score,Total Score
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,ROG.SW,2.48,11.54,9.52,23.53
3,MC.PA,0.76,5.93,5.67,12.37
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
352,NaN,NaN,NaN,NaN,NaN
353,NaN,NaN,NaN,NaN,NaN
354,NaN,NaN,NaN,NaN,NaN
355,NaN,NaN,NaN,NaN,NaN


In [15]:
# count missing esg scores
print(esg_scores.isna().sum())

Ticker Yahoo         175
Environment Score    175
Social Score         175
Governance Score     175
Total Score          175
dtype: int64


In [12]:
# concat the two dataframes
df = pd.concat([df, esg_scores], axis=1)

In [14]:
# save the dataframe
df.to_excel('./datas/MSCIeurope.xlsx')

### create the datasets with the price for all ticker

In [7]:
tickers = yf.Tickers(df.loc[(df['Ticker Yahoo'].notna()), 'Ticker Yahoo'].tolist())
tickers_list = df.loc[(df['Ticker Yahoo'].notna()), 'Ticker Yahoo'].tolist()

In [8]:
data = yf.download(
    tickers=tickers_list,
    threads=True,
    group_by='ticker',
 
)

[*********************100%***********************]  288 of 288 completed

2 Failed downloads:
- SEK=F: 1d data not available for startTime=-2208994789 and endTime=1669920542. Only 100 years worth of day granularity data are allowed to be fetched per request.
- CNH=X: 1d data not available for startTime=-2208994789 and endTime=1669920559. Only 100 years worth of day granularity data are allowed to be fetched per request.


In [15]:
data

FBK.MI                                                   IPN.PA  \
              Open    High     Low   Close Adj Close     Volume        Open   
Date                                                                          
1962-01-02     NaN     NaN     NaN     NaN       NaN        NaN         NaN   
1962-01-03     NaN     NaN     NaN     NaN       NaN        NaN         NaN   
1962-01-04     NaN     NaN     NaN     NaN       NaN        NaN         NaN   
1962-01-05     NaN     NaN     NaN     NaN       NaN        NaN         NaN   
1962-01-08     NaN     NaN     NaN     NaN       NaN        NaN         NaN   
...            ...     ...     ...     ...       ...        ...         ...   
2022-11-25  15.170  15.365  15.150  15.365    15.365  1166639.0  109.900002   
2022-11-28  15.290  15.370  15.065  15.200    15.200  1161057.0  109.900002   
2022-11-29  15.275  15.300  14.965  15.120    15.120  1553091.0  109.800003   
2022-11-30     NaN     NaN     NaN     NaN       NaN        NaN         NaN   
2022-12-01  15.520  15.625  15.330  15.450    15.450  2384112.0  108.000000   

                                                ...          SEB               \
                  High         Low       Close  ...          Low        Close   
Date                                            ...                             
1962-01-02         NaN         NaN         NaN  ...          NaN          NaN   
1962-01-03         NaN         NaN         NaN  ...          NaN          NaN   
1962-01-04         NaN         NaN         NaN  ...          NaN          NaN   
1962-01-05         NaN         NaN         NaN  ...          NaN          NaN   
1962-01-08         NaN         NaN         NaN  ...          NaN          NaN   
...                ...         ...         ...  ...          ...          ...   
2022-11-25  110.900002  109.400002  109.900002  ...  3875.000000  3999.989990   
2022-11-28  110.099998  108.000000  109.300003  ...  3929.870117  3989.979980   
2022-11-29  110.000000  108.099998  108.800003  ...  3940.959961  3940.979980   
2022-11-30         NaN         NaN         NaN  ...  3928.510010  3952.850098   
2022-12-01  108.000000  106.300003  107.199997  ...  3945.000000  4013.879883   

                                      AIR                                   \
              Adj Close Volume       Open       High        Low      Close   
Date                                                                         
1962-01-02          NaN    NaN        NaN        NaN        NaN        NaN   
1962-01-03          NaN    NaN        NaN        NaN        NaN        NaN   
1962-01-04          NaN    NaN        NaN        NaN        NaN        NaN   
1962-01-05          NaN    NaN        NaN        NaN        NaN        NaN   
1962-01-08          NaN    NaN        NaN        NaN        NaN        NaN   
...                 ...    ...        ...        ...        ...        ...   
2022-11-25  3999.989990  200.0  46.470001  46.669998  46.139999  46.549999   
2022-11-28  3989.979980  400.0  46.040001  46.290001  44.830002  44.980000   
2022-11-29  3940.979980  600.0  45.000000  45.419998  44.939999  45.299999   
2022-11-30  3952.850098  500.0  45.209999  46.580002  44.770000  46.560001   
2022-12-01  4013.879883  182.0  46.700001  46.990002  46.320000  46.770000   

                                 
            Adj Close    Volume  
Date                             
1962-01-02        NaN       NaN  
1962-01-03        NaN       NaN  
1962-01-04        NaN       NaN  
1962-01-05        NaN       NaN  
1962-01-08        NaN       NaN  
...               ...       ...  
2022-11-25  46.549999   81800.0  
2022-11-28  44.980000  102700.0  
2022-11-29  45.299999  103000.0  
2022-11-30  46.560001  230600.0  
2022-12-01  46.770000   39825.0  

[15607 rows x 1728 columns]

In [29]:
# filter by close price anr remove the multiindex
columns = pd.MultiIndex.from_product([tickers_list, ['Close']])
data_filter = data[columns]
data_filter.columns = data_filter.columns.droplevel(1)
# from 2015 only 
data_from = pd.Timestamp('2015-01-01')
data_filter = data_filter.loc[data_from:]
# to csv
data_filter.to_csv('./datas/historical_prices.csv')